# 22.4. Автоматизация с параметризацией для негативных тестов
Выделим из теста, который мы написали в прошлом юните, негативные кейсы и оформим их отдельным тестом:

In [ ]:
@pytest.mark.parametrize("name"
   , [generate_string(255), generate_string(1001), russian_chars(), russian_chars().upper(), chinese_chars(),
      special_chars(), '123']
   , ids=['255 symbols', 'more than 1000 symbols', 'russian', 'RUSSIAN', 'chinese', 'specials', 'digit'])
@pytest.mark.parametrize("animal_type"
   , [generate_string(255), generate_string(1001), russian_chars(), russian_chars().upper(), chinese_chars(),
      special_chars(), '123']
   , ids=['255 symbols', 'more than 1000 symbols', 'russian', 'RUSSIAN', 'chinese', 'specials', 'digit'])
@pytest.mark.parametrize("age", ['1'], ids=['min'])
def test_add_new_pet_simple(name, animal_type, age):
   """Проверяем, что можно добавить питомца с различными данными"""

   # Добавляем питомца
   pytest.status, result = pf.add_new_pet_simple(pytest.key, name, animal_type, age)

   # Сверяем полученный ответ с ожидаемым результатом
   assert pytest.status == 200
   assert result['name'] == name
   assert result['age'] == age
   assert result['animal_type'] == animal_type


@pytest.mark.parametrize("name", [''], ids=['empty'])
@pytest.mark.parametrize("animal_type", [''], ids=['empty'])
@pytest.mark.parametrize("age",
                        ['', '-1', '0', '100', '1.5', '2147483647', '2147483648', special_chars(), russian_chars(),
                         russian_chars().upper(), chinese_chars()]
   , ids=['empty', 'negative', 'zero', 'greater than max', 'float', 'int_max', 'int_max + 1', 'specials',
          'russian', 'RUSSIAN', 'chinese'])
def test_add_new_pet_simple_negative(name, animal_type, age):

   # Добавляем питомца
   pytest.status, result = pf.add_new_pet_simple(pytest.key, name, animal_type, age)

   # Сверяем полученный ответ с ожидаемым результатом
   assert pytest.status == 400

Что изменилось?
1
Во-первых, теперь мы имеем отдельным тестом негативный набор и, как следствие, мы больше не нуждаемся в функции is_age_valid(age). Смело можем убрать её.

2
Во-вторых, мы чётко видим, какие значения каких параметров теста вызывают ошибку при обращении с ними в сервис, то есть результаты тестов теперь нагляднее читать из-за того, что мы категоризировали набор.

3
В-третьих, если вдруг разработчик решит добавить какие-либо параметры в сущность питомца (либо удалить их из неё, либо изменить набор значений, при которых сервер будет считать параметр валидным), то нам не придётся переписывать блок кода, который отвечает за выбор проверок в тесте, как мы делали бы это раньше.